# Advent of code 2022

In [1]:
import pandas as pd

import pprint

## Part 1

In [2]:
test_input='''30373
25512
65332
33549
35390'''


In [3]:
with open('data/day08.txt') as fIn:
    puzzle_input=fIn.read()

Put the data into a DataFrame:

In [4]:
df=pd.DataFrame([[int(i) for i in nl] for nl in test_input.splitlines()])
df

,0,1,2,3,4
0,3,0,3,7,3
1,2,5,5,1,2
2,6,5,3,3,2
3,3,3,5,4,9
4,3,5,3,9,0


A non-edge tree is hidden if there is a tree either side in the row which is (non-strictly) taller than it:

In [5]:
def hidden_in_series(ss_in):
    
    return pd.Series({ss_in.index[i]:((ss_in.iloc[i]<=ss_in.iloc[:i]).any()) and ((ss_in.iloc[i]<=ss_in.iloc[i+1:]).any())
                      for (i, v) in enumerate(ss_in)})

Helpfully, use of `.any()` sets the first and last values to `False`.

And extend this to the whole DataFrame:

In [6]:
def hidden_in_dataframe(df_in):
    visible_vertically_df=df_in.apply(hidden_in_series)
    
    # And use the transpose to get the horizontal view
    visible_horizontally_df=df_in.T.apply(hidden_in_series).T
    
    # And just use 'and' to get the final result:

    return visible_horizontally_df & visible_vertically_df

In [7]:
h_df=hidden_in_dataframe(df)
h_df

,0,1,2,3,4
0,False,False,False,False,False
1,False,False,False,True,False
2,False,False,True,False,False
3,False,True,False,True,False
4,False,False,False,False,False


And just count up the number of `False`s:

In [8]:
def day08_a(str_in):
    h_df=hidden_in_dataframe(pd.DataFrame([[int(i) for i in nl]
                                           for nl in str_in.splitlines()]))
    
    
    
    return sum(h_df.apply(lambda s:s.value_counts()[False]))

In [9]:
assert day08_a(test_input)==21

In [10]:
day08_a(puzzle_input)

1690

## Part 2

Bit hacky to convert (back) to a list, but it should make things fairly straightforward. Given a series and an index, return the number of trees which can be seen in either direction from the indexed tree.

Actually, it's a bit more fiddly than it needs to be because if the view's blocked we count the blocking tree, but the edge needs to be handled slightly differently:

In [11]:
def scenic_score(ss_in, idx):
    
    # Count trees forward
    l=list(ss_in.loc[idx:])
    i=0
    for t in l[1:]:
        i+=1
        if t>=l[0]:
            break

    # Count trees backward
    l=list(ss_in.loc[:idx])
    l.reverse()
    j=0
    for t in l[1:]:
        j+=1
        if t>=l[0]:
            break

    return i*j

In [12]:
def scenic_score_series(ss_in):
    return pd.Series({i:scenic_score(ss_in, i) for i in ss_in.index})

In [13]:
scenic_score_series(df[2])

0    0
1    2
2    1
3    2
4    0
dtype: int64

Then calculate for a whole dataframe:

In [14]:
df.apply(scenic_score_series) * df.T.apply(scenic_score_series).T

,0,1,2,3,4
0,0,0,0,0,0
1,0,1,4,1,0
2,0,6,1,2,0
3,0,1,8,3,0
4,0,0,0,0,0


OK, that seems to do the trick. Now put into a function:


In [15]:
def day08_b(str_in):
    df=pd.DataFrame([[int(i) for i in nl] 
                     for nl in str_in.splitlines()])
    
    scenic_scores_df=df.apply(scenic_score_series) * df.T.apply(scenic_score_series).T
    
    return scenic_scores_df.max().max()

In [16]:
assert day08_b(test_input)==8

In [17]:
day08_b(puzzle_input)

535680